In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import pandas as pd
import ast
writer = pd.ExcelWriter('rugby_stats_for_clubs_and_teams.xlsx', engine='xlsxwriter')
writer.save()
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql.setTimeout(5000)

DBQuery to retrieve stats about rugby players

In [3]:
def get_json_rugby_team_front_row_data(league):
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname (MAX(?finishingPosition) as ?finishingPosition) (concat('[',group_concat(distinct ?longitude;separator=','),']') as ?longitude) (concat('[',group_concat(distinct ?latitude;separator=','),']') as ?latitude) (AVG(year(xsd:dateTime(?propsbirthDate))) as ?propsbirthDate) (AVG(?propsheight) as ?propsheight) (AVG(?propsweight) as ?propsweight) (AVG(year(xsd:dateTime(?hookersbirthDate))) as ?hookersbirthDate) (AVG(?hookersweight) as ?hookersweight) (AVG(?hookersheight) as ?hookersheight) (AVG(?height) as ?height) (AVG(?weight) as ?weight) (MAX(year(xsd:dateTime(?birthDate))) as ?youngestBirthYear) (MIN(year(xsd:dateTime(?birthDate))) as ?oldestBirthYear) (AVG(year(xsd:dateTime(?birthDate))) as ?averageBirthAge) 
            
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?team dbo:location ?location .
        ?location geo:lat ?latitude .
        ?location geo:long ?longitude .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:name ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        ?props <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_props> .
        ?props p:ruProclubs|p:clubs ?team .
        ?props dbo:height ?propsheight .
        ?props dbo:weight ?propsweight .
        ?props dbp:birthDate ?propsbirthDate .
        ?hookers <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_hookers> .
        ?hookers p:ruProclubs|p:clubs ?team .
        ?hookers dbo:height ?hookersheight .
        ?hookers dbo:weight ?hookersweight .
        ?hookers dbp:birthDate ?hookersbirthDate .
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']

def get_json_rugby_team_lock_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname  (AVG(year(xsd:dateTime(?locksbirthDate))) as ?locksbirthDate) (concat('["',group_concat(distinct ?comps;separator='","'),'"]') as ?comps) (AVG(?locksweight) as ?locksweight) (AVG(?locksheight) as ?locksheight) 
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:name ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        ?locks <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_locks> .
        ?locks p:ruProclubs|p:clubs ?team .
        ?locks dbo:height ?locksheight .
        ?locks dbo:weight ?locksweight .
        ?locks dbp:birthDate ?locksbirthDate .
        OPTIONAL {?comps dbp:champions ?team} .
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?locksbirthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']

def get_json_rugby_team_back_row_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname (MAX(?finishingPosition) as ?finishingPosition)  (AVG(year(xsd:dateTime(?flankersbirthDate))) as ?flankersbirthDate) (AVG(?flankersweight) as ?flankersweight) (AVG(?flankersheight) as ?flankersheight) (AVG(year(xsd:dateTime(?number_eightsbirthDate))) as ?number_eightsbirthDate) (AVG(?number_eightsweight) as ?number_eightsweight) (AVG(?number_eightsheight) as ?number_eightsheight)  (AVG(?height) as ?height) (AVG(?weight) as ?weight) (MAX(year(xsd:dateTime(?birthDate))) as ?youngestBirthYear) (MIN(year(xsd:dateTime(?birthDate))) as ?oldestBirthYear) (AVG(year(xsd:dateTime(?birthDate))) as ?averageBirthAge)      
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:birthName ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        ?flankers <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_flankers> .
        ?flankers p:ruProclubs|p:clubs ?team .
        ?flankers dbo:height ?flankersheight .
        ?flankers dbo:weight ?flankersweight .
        ?flankers dbp:birthDate ?flankersbirthDate .
        ?number_eights <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_number_eights> .
        ?number_eights p:ruProclubs|p:clubs ?team .
        ?number_eights dbo:height ?number_eightsheight .
        ?number_eights dbo:weight ?number_eightsweight .
        ?number_eights dbp:birthDate ?number_eightsbirthDate .
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']


def get_json_rugby_team_half_backs_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname (AVG(year(xsd:dateTime(?scrum_halvesbirthDate))) as ?scrum_halvesbirthDate) (AVG(?scrum_halvesweight) as ?scrum_halvesweight) (AVG(?scrum_halvesheight) as ?scrum_halvesheight) (AVG(year(xsd:dateTime(?fly_halvesbirthDate))) as ?fly_halvesbirthDate) (AVG(?fly_halvesweight) as ?fly_halvesweight) (AVG(?fly_halvesheight) as ?fly_halvesheight) 
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:birthName ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        
        ?scrum_halves <http://purl.org/dc/terms/subject>  <http://dbpedia.org/resource/Category:Rugby_union_scrum-halves> .
        ?scrum_halves p:ruProclubs|p:clubs ?team .
        ?scrum_halves dbo:height ?scrum_halvesheight .
        ?scrum_halves dbo:weight ?scrum_halvesweight .
        ?scrum_halves dbp:birthDate ?scrum_halvesbirthDate .
        
        ?fly_halves <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_fly-halves> .
        ?fly_halves p:ruProclubs|p:clubs ?team .
        ?fly_halves dbo:height ?fly_halvesheight .
        ?fly_halves dbo:weight ?fly_halvesweight .
        ?fly_halves dbp:birthDate ?fly_halvesbirthDate .
        
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']


def get_json_rugby_team_centres_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname (AVG(year(xsd:dateTime(?centresbirthDate))) as ?centresbirthDate) (AVG(?centresweight) as ?centresweight) (AVG(?centresheight) as ?centresheight) 
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:name ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        
        ?centres <http://purl.org/dc/terms/subject>  <http://dbpedia.org/resource/Category:Rugby_union_centres> .
        ?centres p:ruProclubs|p:clubs ?team .
        ?centres dbo:height ?centresheight .
        ?centres dbo:weight ?centresweight .
        ?centres dbp:birthDate ?centresbirthDate .
        
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']


def get_json_rugby_team_back_three_data(league):
    #Final Working version to get teams
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?teamname (AVG(year(xsd:dateTime(?wingersbirthDate))) as ?wingersbirthDate) (AVG(?wingersweight) as ?wingersweight) (AVG(?wingersheight) as ?wingersheight) (AVG(year(xsd:dateTime(?full_backsbirthDate))) as ?full_backsbirthDate) (AVG(?full_backsweight) as ?full_backsweight) (AVG(?full_backsheight) as ?full_backsheight)
    WHERE {
        ?team p:league """ + league + """ .
        ?team dbp:teamname ?teamname .
        ?team dbp:position ?finishingPosition .
        ?players p:ruProclubs|p:clubs ?team .
        ?players dbp:name ?name .
        ?players dbo:height ?height .
        ?players dbo:weight ?weight .
        ?players dbp:birthDate ?birthDate .
        
        ?wingers <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_wings> .
        ?wingers p:ruProclubs|p:clubs ?team .
        ?wingers dbo:height ?wingersheight .
        ?wingers dbo:weight ?wingersweight .
        ?wingers dbp:birthDate ?wingersbirthDate .
        
        ?full_backs <http://purl.org/dc/terms/subject> <http://dbpedia.org/resource/Category:Rugby_union_fullbacks> .
        ?full_backs p:ruProclubs|p:clubs ?team .
        ?full_backs dbo:height ?full_backsheight .
        ?full_backs dbo:weight ?full_backsweight .
        ?full_backs dbp:birthDate ?full_backsbirthDate .
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>) .
        FILTER(datatype(?finishingPosition) = <http://www.w3.org/2001/XMLSchema#integer>)
     }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']


In [4]:
prem_rugby_player_front_row_data = get_json_rugby_team_front_row_data("<http://dbpedia.org/resource/Premiership_Rugby>")

In [5]:

prem_rugby_player_lock_data = get_json_rugby_team_lock_data("<http://dbpedia.org/resource/Premiership_Rugby>")


In [6]:
prem_rugby_player_back_row_data = get_json_rugby_team_back_row_data("<http://dbpedia.org/resource/Premiership_Rugby>")

In [7]:
prem_rugby_player_half_backs_data = get_json_rugby_team_half_backs_data("<http://dbpedia.org/resource/Premiership_Rugby>")


In [8]:
prem_rugby_player_centres_data = get_json_rugby_team_centres_data("<http://dbpedia.org/resource/Premiership_Rugby>")

In [9]:
prem_rugby_player_back_three_data = get_json_rugby_team_back_three_data("<http://dbpedia.org/resource/Premiership_Rugby>")

In [10]:
Top_14_rugby_player_front_row_data = get_json_rugby_team_front_row_data("<http://dbpedia.org/resource/Top_14>")

In [11]:
Top_14_rugby_player_lock_data = get_json_rugby_team_lock_data("<http://dbpedia.org/resource/Top_14>")

In [12]:
Top_14_rugby_player_back_row_data = get_json_rugby_team_back_row_data("<http://dbpedia.org/resource/Top_14>")

In [13]:
Top_14_rugby_player_half_backs_data = get_json_rugby_team_half_backs_data("<http://dbpedia.org/resource/Top_14>")

In [14]:
Top_14_rugby_player_centres_data = get_json_rugby_team_centres_data("<http://dbpedia.org/resource/Top_14>")

In [15]:
Top_14_rugby_player_back_three_data = get_json_rugby_team_back_three_data("<http://dbpedia.org/resource/Top_14>")

In [16]:
Pro14_rugby_player_front_row_data = get_json_rugby_team_front_row_data("<http://dbpedia.org/resource/Pro14>")

In [17]:
Pro14_rugby_player_lock_data = get_json_rugby_team_lock_data("<http://dbpedia.org/resource/Pro14>")

In [18]:
Pro14_rugby_player_back_row_data = get_json_rugby_team_back_row_data("<http://dbpedia.org/resource/Pro14>")

In [19]:
Pro14_rugby_player_half_backs_data = get_json_rugby_team_half_backs_data("<http://dbpedia.org/resource/Pro14>")

In [20]:
Pro14_rugby_player_centres_data = get_json_rugby_team_centres_data("<http://dbpedia.org/resource/Pro14>")

In [21]:
Pro14_rugby_player_back_three_data = get_json_rugby_team_back_three_data("<http://dbpedia.org/resource/Pro14>")

In [22]:
Super_Rugby_rugby_player_front_row_data = get_json_rugby_team_front_row_data("<http://dbpedia.org/resource/Super_Rugby>")

In [23]:
Super_Rugby_rugby_player_lock_data = get_json_rugby_team_lock_data("<http://dbpedia.org/resource/Super_Rugby>")

In [24]:
Super_Rugby_rugby_player_back_row_data = get_json_rugby_team_back_row_data("<http://dbpedia.org/resource/Super_Rugby>")

In [25]:
Super_Rugby_rugby_player_half_backs_data = get_json_rugby_team_half_backs_data("<http://dbpedia.org/resource/Super_Rugby>")

In [26]:
Super_Rugby_rugby_player_centres_data = get_json_rugby_team_centres_data("<http://dbpedia.org/resource/Super_Rugby>")

In [27]:
Super_Rugby_rugby_player_back_three_data = get_json_rugby_team_back_three_data("<http://dbpedia.org/resource/Super_Rugby>")

In [30]:
def get_json_player_data(position):
     #Version to get all Rugby Players
    #Trying to get the unqi
    q1 = """
    PREFIX p: <http://dbpedia.org/property/> 
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT DISTINCT (MIN(?birthName) as ?birthName) (concat('[',group_concat(distinct ?longitude;separator=','),']') as ?longitude) (concat('[',group_concat(distinct ?latitude;separator=','),']') as ?latitude) (concat('["',group_concat(distinct ?teamname;separator='","'),'"]') as ?teamname) (MIN(?population) as ?population) (SUM(?reppoints) as ?reppoints)  (concat('[',group_concat(distinct ?repcap;separator=','),']') as ?repcap) (concat('[',group_concat(distinct ?clubcap;separator=','),']') as ?clubcap) ?height ?weight (year(xsd:dateTime(?birthDate)) as ?birthDate)        
    WHERE {
        ?player <http://purl.org/dc/terms/subject> """ + position + """ .
        ?player dbp:name|dbp:birthName ?birthName .
        ?player dbo:height ?height .
        ?player dbo:weight ?weight .
        ?player dbp:apps|dbp:ruClubcaps ?clubcap .
        ?player dbp:birthDate ?birthDate .
        ?player p:ruProclubs|p:clubs ?team .
        ?player dbp:birthPlace ?birthPlace .
        ?birthPlace dbo:populationTotal ?population .
        ?birthPlace geo:lat ?latitude .
        ?birthPlace geo:long ?longitude .
        ?team dbp:teamname ?teamname .
        OPTIONAL {?player dbp:repcaps|dbp:ruNationalcaps ?repcap . }
        OPTIONAL {?player dbp:dbp:reppoints ?reppoints . }
        FILTER(datatype(?birthDate) = <http://www.w3.org/2001/XMLSchema#date>)
        FILTER(datatype(?repcap) = <http://www.w3.org/2001/XMLSchema#integer>)
        FILTER(datatype(?clubcap) = <http://www.w3.org/2001/XMLSchema#integer>)
        FILTER(xsd:date(?birthDate) > "1985-01-01"^^xsd:date)
    }"""
    sparql.setQuery(q1)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    return qres['results']['bindings']

In [31]:
props = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_props>")

In [32]:
hookers = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_hookers>")

In [33]:
locks = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_locks>")

In [34]:
wing_forwards = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_flankers>")

In [35]:
number_8s = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_number_eights>")

In [36]:
scrum_halves = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_scrum-halves>")

In [37]:
fly_halves = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_fly-halves>")

In [38]:
centres = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_centres>")

In [39]:
wingers = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_wings>")

In [40]:
fullbacks = get_json_player_data("<http://dbpedia.org/resource/Category:Rugby_union_fullbacks>")

In [41]:
def check_if_team_already_in_dataset(dct,name):
    player_already_present = False
    for names in dct["teamname"]:
        if names == name:
            player_already_present = True
    return player_already_present
    
def get_all_teams(prem, urc, superRugby,top14):
    #Include longitude and latitude of the rugby playing countries according to dbpedia(France,Ireland,England,Wales,Scotland,Italy,South Africa, New Zealand, Australia,Japan, Georgia,Tonga,Samoa,Fiji)
    longitude_of_countries = [2.000000, 2.350000, -8.000000,-0.116667,-3.183333,-3.188889,12.000000,12.483334,25.000000,173.000000,174.783340,149.124451,139.766663,43.500000,44.783333,-175.199997,-175.000000,-172.333328,-171.750000,178.449997]
    latitude_of_countries = [47.000000, 48.849998, 53.416668,51.500000,51.483334,55.953056,41.900002,43.000000,-30.000000,-42.000000,-41.299999,-35.308056,35.683334,41.716667,42.000000,-21.133333,-20.000000,-13.833333,-13.583333,-18.166666]
    club_attributes_dct = {"competitionsWon":[],"latitude":[],"longitude":[],"propsheight":[], "propsweight":[], "propsbirthDate":[], "hookersheight":[], "hookersweight":[], "hookersbirthDate":[], "locksheight":[], "locksweight":[], "locksbirthDate":[], "flankersheight":[], "flankersweight":[], "flankersbirthDate":[], "number_eightsheight":[], "number_eightsweight":[], "number_eightsbirthDate":[], "scrum_halvesheight":[], "scrum_halvesweight":[], "scrum_halvesbirthDate":[], "fly_halvesheight":[], "fly_halvesweight":[], "fly_halvesbirthDate":[], "centresheight":[], "centresweight":[], "centresbirthDate":[], "wingersheight":[], "wingersweight":[], "wingersbirthDate":[], "full_backsheight":[], "full_backsweight":[], "full_backsbirthDate":[],"teamname":[], "LeagueName":[], "height":[], "weight":[], "finishingPosition":[],"youngestBirthYear":[], "oldestBirthYear":[], "averageBirthAge":[]}
    leagues = [prem, urc, superRugby,top14]
    type_string = ["prem", "urc", "superRugby","top14" ]
    index = 0
    team_index = 0
    for league in leagues:
        team_index =0
        for team in league[0]:     
            club_attributes_dct["teamname"].append(team['teamname']['value'])
            club_attributes_dct["LeagueName"].append(type_string[index])
            club_attributes_dct["finishingPosition"].append(team['finishingPosition']['value'])
            club_attributes_dct["height"].append(team['height']['value'])
            club_attributes_dct["weight"].append(team['weight']['value'])
            
            #Find only the longitude and latitude of the province or county instead of country
            longitudes = ast.literal_eval(team['longitude']['value'])
            latitudes = ast.literal_eval(team['latitude']['value'])
            club_latitude = False
            club_longitude = False
            
            for latitude in latitudes:
                if latitude not in latitude_of_countries:
                    club_latitude = latitude
            
            if club_latitude == False:
                club_attributes_dct["latitude"].append(latitudes[0])
            
            else:
                club_attributes_dct["latitude"].append(club_latitude)
                
            for longitude in longitudes:
                if longitude not in longitude_of_countries:
                    club_longitude = longitude
      
            if club_longitude == False:
                club_attributes_dct["longitude"].append(longitudes[0])
            else:
                club_attributes_dct["longitude"].append(club_longitude)
                
            
            club_attributes_dct["youngestBirthYear"].append(team['youngestBirthYear']['value'])
            club_attributes_dct["oldestBirthYear"].append(team['oldestBirthYear']['value'])
            club_attributes_dct["averageBirthAge"].append(team['averageBirthAge']['value'])
            
            club_attributes_dct["propsheight"].append(team['propsheight']['value'])
            club_attributes_dct["propsweight"].append(team['propsweight']['value'])
            club_attributes_dct["propsbirthDate"].append(team['propsbirthDate']['value'])
            
            club_attributes_dct["hookersheight"].append(team['hookersheight']['value'])
            club_attributes_dct["hookersweight"].append(team['hookersweight']['value'])
            club_attributes_dct["hookersbirthDate"].append(team['hookersbirthDate']['value'])

            i_loop = 0
            team_present = False
            if league[1][0]['teamname']['value'] == team['teamname']['value']:
                club_attributes_dct["locksheight"].append(league[1][i_loop]['locksheight']['value'])
                club_attributes_dct["locksweight"].append(league[1][i_loop]['locksweight']['value'])
                club_attributes_dct["locksbirthDate"].append(league[1][i_loop]['locksbirthDate']['value'])
                club_attributes_dct["competitionsWon"].append(len(ast.literal_eval(league[1][i_loop]['comps']['value'])))

            else:
                while league[1][i_loop]['teamname']['value'] != team['teamname']['value'] and len(league[1]) > (i_loop + 1):
                    i_loop +=1
                    if league[1][i_loop]['teamname']['value'] == team['teamname']['value']:
                        team_present = True
                        club_attributes_dct["locksheight"].append(league[1][i_loop]['locksheight']['value'])
                        club_attributes_dct["locksweight"].append(league[1][i_loop]['locksweight']['value'])
                        club_attributes_dct["locksbirthDate"].append(league[1][i_loop]['locksbirthDate']['value'])
                        club_attributes_dct["competitionsWon"].append(len(ast.literal_eval(league[1][i_loop]['comps']['value'])))
                        break
                if team_present == False:
                        club_attributes_dct["locksheight"].append("")
                        club_attributes_dct["locksweight"].append("")
                        club_attributes_dct["locksbirthDate"].append("")  
                        club_attributes_dct["competitionsWon"].append("")

                
            i_loop = 0
            team_present = False
            if league[2][0]['teamname']['value'] == team['teamname']['value']:
                club_attributes_dct["flankersheight"].append(league[2][i_loop]['flankersheight']['value'])
                club_attributes_dct["flankersweight"].append(league[2][i_loop]['flankersweight']['value'])
                club_attributes_dct["flankersbirthDate"].append(league[2][i_loop]['flankersbirthDate']['value'])
                club_attributes_dct["number_eightsheight"].append(league[2][i_loop]['number_eightsheight']['value'])
                club_attributes_dct["number_eightsweight"].append(league[2][i_loop]['number_eightsweight']['value'])
                club_attributes_dct["number_eightsbirthDate"].append(league[2][i_loop]['number_eightsbirthDate']['value'])
            
            else:
                while league[2][i_loop]['teamname']['value'] != team['teamname']['value'] and len(league[2]) > (i_loop + 1):
                    i_loop +=1
                    if league[2][i_loop]['teamname']['value'] == team['teamname']['value']:
                        team_present = True
                        club_attributes_dct["flankersheight"].append(league[2][i_loop]['flankersheight']['value'])
                        club_attributes_dct["flankersweight"].append(league[2][i_loop]['flankersweight']['value'])
                        club_attributes_dct["flankersbirthDate"].append(league[2][i_loop]['flankersbirthDate']['value'])
                        club_attributes_dct["number_eightsheight"].append(league[2][i_loop]['number_eightsheight']['value'])
                        club_attributes_dct["number_eightsweight"].append(league[2][i_loop]['number_eightsweight']['value'])
                        club_attributes_dct["number_eightsbirthDate"].append(league[2][i_loop]['number_eightsbirthDate']['value'])
                        break
                if team_present == False:
                        club_attributes_dct["flankersheight"].append("")
                        club_attributes_dct["flankersweight"].append("")
                        club_attributes_dct["flankersbirthDate"].append("")
                        club_attributes_dct["number_eightsheight"].append("")
                        club_attributes_dct["number_eightsweight"].append("")
                        club_attributes_dct["number_eightsbirthDate"].append("")
                        

                    
            i_loop = 0
            team_present = False
            if league[3][0]['teamname']['value'] == team['teamname']['value']:  
                club_attributes_dct["scrum_halvesheight"].append(league[3][i_loop]['scrum_halvesheight']['value'])
                club_attributes_dct["scrum_halvesweight"].append(league[3][i_loop]['scrum_halvesweight']['value'])
                club_attributes_dct["scrum_halvesbirthDate"].append(league[3][i_loop]['scrum_halvesbirthDate']['value'])
                club_attributes_dct["fly_halvesheight"].append(league[3][i_loop]['fly_halvesheight']['value'])
                club_attributes_dct["fly_halvesweight"].append(league[3][i_loop]['fly_halvesweight']['value'])
                club_attributes_dct["fly_halvesbirthDate"].append(league[3][i_loop]['fly_halvesbirthDate']['value'])
            else:
                while league[3][i_loop]['teamname']['value'] != team['teamname']['value'] and len(league[3]) > (i_loop + 1):
                    i_loop +=1
                    if league[3][i_loop]['teamname']['value'] == team['teamname']['value']:
                        team_present = True
                        club_attributes_dct["scrum_halvesheight"].append(league[3][i_loop]['scrum_halvesheight']['value'])
                        club_attributes_dct["scrum_halvesweight"].append(league[3][i_loop]['scrum_halvesweight']['value'])
                        club_attributes_dct["scrum_halvesbirthDate"].append(league[3][i_loop]['scrum_halvesbirthDate']['value'])
                        club_attributes_dct["fly_halvesheight"].append(league[3][i_loop]['fly_halvesheight']['value'])
                        club_attributes_dct["fly_halvesweight"].append(league[3][i_loop]['fly_halvesweight']['value'])
                        club_attributes_dct["fly_halvesbirthDate"].append(league[3][i_loop]['fly_halvesbirthDate']['value'])
                        break
                if team_present == False:
                    club_attributes_dct["scrum_halvesheight"].append("")
                    club_attributes_dct["scrum_halvesweight"].append("")
                    club_attributes_dct["scrum_halvesbirthDate"].append("")
                    club_attributes_dct["fly_halvesheight"].append("")
                    club_attributes_dct["fly_halvesweight"].append("")
                    club_attributes_dct["fly_halvesbirthDate"].append("")
            
            i_loop = 0
            team_present = False
            if league[4][0]['teamname']['value'] == team['teamname']['value']:  
                club_attributes_dct["centresheight"].append(league[4][i_loop]['centresheight']['value'])
                club_attributes_dct["centresweight"].append(league[4][i_loop]['centresweight']['value'])
                club_attributes_dct["centresbirthDate"].append(league[4][i_loop]['centresbirthDate']['value'])
            else:
                while league[4][i_loop]['teamname']['value'] != team['teamname']['value'] and len(league[4]) > (i_loop + 1):
                    i_loop +=1
                    if league[4][i_loop]['teamname']['value'] == team['teamname']['value']:
                        club_attributes_dct["centresheight"].append(league[4][i_loop]['centresheight']['value'])
                        club_attributes_dct["centresweight"].append(league[4][i_loop]['centresweight']['value'])
                        club_attributes_dct["centresbirthDate"].append(league[4][i_loop]['centresbirthDate']['value'])
                        team_present = True
                        break
                if team_present == False:
                    club_attributes_dct["centresheight"].append("")
                    club_attributes_dct["centresweight"].append("")
                    club_attributes_dct["centresbirthDate"].append("")
            i_loop = 0
            team_present = False
            if league[5][0]['teamname']['value'] == team['teamname']['value']: 
                club_attributes_dct["wingersheight"].append(league[5][i_loop]['wingersheight']['value'])
                club_attributes_dct["wingersweight"].append(league[5][i_loop]['wingersweight']['value'])
                club_attributes_dct["wingersbirthDate"].append(league[5][i_loop]['wingersbirthDate']['value'])
                club_attributes_dct["full_backsheight"].append(league[5][i_loop]['full_backsheight']['value'])
                club_attributes_dct["full_backsweight"].append(league[5][i_loop]['full_backsweight']['value'])
                club_attributes_dct["full_backsbirthDate"].append(league[5][i_loop]['full_backsbirthDate']['value'])
            
            else:
                while league[5][i_loop]['teamname']['value'] != team['teamname']['value'] and len(league[5]) > (i_loop + 1):
                    i_loop +=1
                    if league[5][i_loop]['teamname']['value'] == team['teamname']['value']:
                        club_attributes_dct["wingersheight"].append(league[5][i_loop]['wingersheight']['value'])
                        club_attributes_dct["wingersweight"].append(league[5][i_loop]['wingersweight']['value'])
                        club_attributes_dct["wingersbirthDate"].append(league[5][i_loop]['wingersbirthDate']['value'])
                        club_attributes_dct["full_backsheight"].append(league[5][i_loop]['full_backsheight']['value'])
                        club_attributes_dct["full_backsweight"].append(league[5][i_loop]['full_backsweight']['value'])
                        club_attributes_dct["full_backsbirthDate"].append(league[5][i_loop]['full_backsbirthDate']['value'])
                        team_present = True
                        break
                    
                if team_present == False:
                    club_attributes_dct["wingersheight"].append("")
                    club_attributes_dct["wingersweight"].append("")
                    club_attributes_dct["wingersbirthDate"].append("")
                    club_attributes_dct["full_backsheight"].append("")
                    club_attributes_dct["full_backsweight"].append("")
                    club_attributes_dct["full_backsbirthDate"].append("")
            team_index +=1            
        index += 1
    return club_attributes_dct

def check_if_player_already_in_dataset(dct,name):
    player_already_present = False
    for names in dct["birthName"]:
        if names == name:
            player_already_present = True
    return player_already_present
    

def get_all_players(props,hookers,locks,wing_forwards,number_8s,scrum_halves,fly_halves,centres,wingers,fullbacks):
    player_attributes_dct = {"birthName":[],"teamnames":[], "numClubsPlayedFor":[], "homePopulation":[],"homeLongitude":[],"homeLatitude":[],"reppoints":[],"Position":[], "height":[], "weight":[], "birthDate":[], "clubcap":[], "repcaps":[]}
    positions = [props,hookers,locks,wing_forwards,number_8s,scrum_halves,fly_halves,centres,wingers,fullbacks]
    type_string = ["props","hookers","locks","wing_forwards","number_8s","scrum_halves","fly_halves","centres","wingers","fullbacks"]
    longitude_of_countries = [2.000000, 2.350000, -8.000000,-0.116667,-3.183333,-3.188889,12.000000,12.483334,25.000000,173.000000,174.783340,149.124451,139.766663,43.500000,44.783333,-175.199997,-175.000000,-172.333328,-171.750000,178.449997]
    latitude_of_countries = [47.000000, 48.849998, 53.416668,51.500000,51.483334,55.953056,41.900002,43.000000,-30.000000,-42.000000,-41.299999,-35.308056,35.683334,41.716667,42.000000,-21.133333,-20.000000,-13.833333,-13.583333,-18.166666]

    index = 0

    for position in positions:
        for player in position:
            if check_if_player_already_in_dataset(player_attributes_dct,player['birthName']['value']) == False:
                longitudes = ast.literal_eval(player['longitude']['value'])
                latitudes = ast.literal_eval(player['latitude']['value'])
                hometown_latitude = False
                hometown_longitude = False
                for latitude in latitudes:
                    if latitude not in latitude_of_countries:
                        hometown_latitude = latitude

                if hometown_latitude == False:
                    player_attributes_dct["homeLatitude"].append(latitudes[0])
                else:
                    player_attributes_dct["homeLatitude"].append(hometown_latitude)

                for longitude in longitudes:
                    if longitude not in longitude_of_countries:
                        hometown_longitude = longitude

                if hometown_longitude == False:
                    player_attributes_dct["homeLongitude"].append(longitudes[0])
                else:
                    player_attributes_dct["homeLongitude"].append(hometown_longitude)
                
                player_attributes_dct["homePopulation"].append(player['population']['value'])
                player_attributes_dct["teamnames"].append(player['teamname']['value'])
                player_attributes_dct["numClubsPlayedFor"].append(len(ast.literal_eval(player['teamname']['value'])))
                player_attributes_dct["birthName"].append(player['birthName']['value'])
                #club_attributes_dct["foundingYear"].append(team['foundingYear']['value'])
                player_attributes_dct["Position"].append(type_string[index])
                #player_attributes_dct["finishingPosition"].append(ast.literal_eval(player['finishingPosition']['value'])[0])
                #club_attributes_dct["stadiumCapacity"].append(team['stadiumCapacity']['value'])
                player_attributes_dct["height"].append(player['height']['value'])
                player_attributes_dct["weight"].append(player['weight']['value'])
                player_attributes_dct["birthDate"].append(player['birthDate']['value'])
                if 'repcap' in player:
                    player_attributes_dct["repcaps"].append(sum(ast.literal_eval(player['repcap']['value'])))
                else:
                    player_attributes_dct["repcaps"].append(0)
                    
                if 'reppoints' in player:
                    player_attributes_dct["reppoints"].append(sum(ast.literal_eval(player['reppoints']['value'])))
                else:
                    player_attributes_dct["reppoints"].append("")
                
                player_attributes_dct["clubcap"].append(sum(ast.literal_eval(player['clubcap']['value'])))
        index += 1
    return player_attributes_dct







In [42]:
prem_rugby_player_data = [prem_rugby_player_front_row_data, prem_rugby_player_lock_data, prem_rugby_player_back_row_data, prem_rugby_player_half_backs_data, prem_rugby_player_centres_data, prem_rugby_player_back_three_data]
urc_rugby_player_data = [Pro14_rugby_player_front_row_data, Pro14_rugby_player_lock_data, Pro14_rugby_player_back_row_data, Pro14_rugby_player_half_backs_data, Pro14_rugby_player_centres_data, Pro14_rugby_player_back_three_data]
super_rugby_player_data = [Super_Rugby_rugby_player_front_row_data, Super_Rugby_rugby_player_lock_data, Super_Rugby_rugby_player_back_row_data, Super_Rugby_rugby_player_half_backs_data, Super_Rugby_rugby_player_centres_data, Super_Rugby_rugby_player_back_three_data]
top14_rugby_player_data = [Top_14_rugby_player_front_row_data, Top_14_rugby_player_lock_data, Top_14_rugby_player_back_row_data, Top_14_rugby_player_half_backs_data, Top_14_rugby_player_centres_data, Top_14_rugby_player_back_three_data]
clubs_all_dct = get_all_teams(prem_rugby_player_data,urc_rugby_player_data,super_rugby_player_data,top14_rugby_player_data)
players_all_dct = get_all_players(props,hookers,locks,wing_forwards,number_8s,scrum_halves,fly_halves,centres,wingers,fullbacks)

In [43]:
import pandas as pd


clubs_all_df = pd.DataFrame(clubs_all_dct)
players_all_df = pd.DataFrame(players_all_dct)

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('rugby_stats_for_clubs_and_teams.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
clubs_all_df.to_excel(writer, sheet_name='Sheet1_Rugby_Teams', index=False)
players_all_df.to_excel(writer, sheet_name='Sheet2_Rugby_Players', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()
    
